# Day 2: Dive!
https://adventofcode.com/2021/day/2

In [2]:
// https://stackoverflow.com/a/35820003

use std::{
    fs::File,
    io::{self, BufRead, BufReader},
    path::Path,
};

fn lines_from_file(filename: impl AsRef<Path>) -> io::Result<Vec<String>> {
    BufReader::new(File::open(filename)?).lines().collect()
}

let lines = lines_from_file("input/day02.txt").expect("Could not load lines");

## Part 1
Calculate the final position after applying the provided moves to the initial position `(0, 0)`.

We implement the moves as a sequence of 2-dimensional vectors which are added to the initial position.

In [3]:
use std::ops;

#[derive(Debug)]
struct Vector2d {
    pub x: i32,
    pub y: i32
}

impl ops::Add<Vector2d> for Vector2d {
    type Output = Vector2d;

    fn add(self, rhs: Vector2d) -> Vector2d {
        Vector2d{x: self.x + rhs.x, y: self.y + rhs.y}
    }
}

// To see how error handling works, we return a Result
fn parse_move(m: &String) -> Result<Vector2d, &'static str> {
    let split_move: Vec<&str> = m.split(" ").collect();
    let direction = split_move[0];
    let speed = split_move[1].parse::<i32>().unwrap();
    
    match direction {
        "forward" => Ok(Vector2d{x: speed, y: 0}),
        "up"      => Ok(Vector2d{x: 0, y: -speed}),
        "down"    => Ok(Vector2d{x: 0, y: speed}),
        _         => Err("could not parse move")
    }
}

In [4]:
let initial_pos = Vector2d{x: 0, y: 0};

// .sum() does not work without implementing another trait, so we use a fold
let final_pos = lines.iter().map(|line| parse_move(line).unwrap()).fold(initial_pos, |sum, v| sum + v);
final_pos.x * final_pos.y

1882980

## Part 2
Now the rules for interpreting the moves are different:
* `down n` decreases the aim by n
* `up n` increases the aim by n
* `forward n` does two things:
    * it increases the horizontal position by `n` units
    * it increases the depth by `n` multiplied by the aim.

In [5]:
// The state is a tuple that consists of
// - horizontal position
// - depth
// - aim
type State = (i32, i32, i32);

Adding vectors with a fold operation is not sufficient any more, so we implement each move as a function that returns the new state.

In [6]:
fn parse_move2(m: &String) -> Box<dyn Fn(State) -> State> {
    let split_move: Vec<&str> = m.split(" ").collect();
    let direction = split_move[0];
    let n = split_move[1].parse::<i32>().unwrap();
    
    match direction {
        "forward" => Box::new(move |(x, depth, aim)| (x + n, depth + n * aim, aim)),
        "up"      => Box::new(move |(x, depth, aim)| (x, depth, aim - n)),
        "down"    => Box::new(move |(x, depth, aim)| (x, depth, aim + n)),
        _         => panic!("Could not parse move")  // simpler error handling in part 2
    }
}

In [7]:
let initial_state = (0, 0, 0);
let final_state = lines.iter().map(|line| parse_move2(line)).fold(initial_state, |state, f| f(state));

final_state.0 * final_state.1

1971232560